In [1]:
import pandas as pd
import numpy as np
import statistics
import math
from datetime import timedelta
import matplotlib.pyplot as plt
import timeit

2020-10-19 17:40:46.306441 - user 9201 - retry 0 - time 0.07220077514648438 
        - returncode 0 
        - command ['/usr/bin/sbatch', '/tmp/condor_g_scratch.0x555e443cfe90.3058472/bl_129ykn']

In [2]:
# with open('../data/slurm_wrapper_ce5.log') as f:
#     data = ([line.split() for line in f if 'sbatch' in line and 'user 9204' in line])
# with open('../data/slurm_wrapper_ce6.log') as f:
#     data.extend([line.split() for line in f if 'sbatch' in line and 'user 9204' in line])
with open('../data/slurm_wrapper_ce5.log') as f:
    data = ([line.split() for line in f])
with open('../data/slurm_wrapper_ce6.log') as f:
    data.extend([line.split() for line in f])

In [3]:
sbatch = pd.DataFrame([[pd.to_datetime(x[0] + 'T' + x[1]), float(x[10]), int(x[13])] 
                       for x in data], columns=['BEGIN', 'DURATION', 'RETURNCODE'])

In [43]:
sbatch['RETURNCODE'].value_counts()

0      8218429
1      1296640
140      24987
255       7347
8           10
Name: RETURNCODE, dtype: int64

In [ ]:
# Check the duration of the process which ran before the one that timed out to see if we can
#  use this to flag the load if it is getting to be too much 
before = []
for index, row in sbatch.iterrows():
    if index and row['RETURNCODE'] == 1:
        before.append(sbatch.iloc[index - 1]['DURATION'])
print(statistics.mean(before))
print(statistics.stdev(before))
print(statistics.median(before))

In [ ]:
plt.boxplot(before)

In [ ]:
count = 0
for index, row in sbatch.iterrows():
    if index and row['RETURNCODE'] == 1 and sbatch.iloc[index - 1]['RETURNCODE'] == 0:
        count += 1
print(count)

In [4]:
sbatch.describe()

,DURATION,RETURNCODE
count,9.547413e+06,9.547413e+06
mean,5.220249e+00,6.984494e-01
std,8.274227e+00,1.004902e+01
min,1.109338e-02,0.000000e+00
25%,1.635637e-01,0.000000e+00
50%,9.148059e-01,0.000000e+00
75%,6.592690e+00,0.000000e+00
max,2.469198e+02,2.550000e+02


In [45]:
dur = [math.modf(x) for x in sbatch['DURATION']]

In [46]:
# next(zip(*test[:10]))

In [47]:
td_input = [[x[1], x[0] * 1000000] for x in dur]
dur_td = [timedelta(seconds=x[0], microseconds=x[1]) for x in td_input]

In [48]:
sbatch['END'] = sbatch['BEGIN'].add(dur_td)

In [49]:

begin_load_df = sbatch[['BEGIN','RETURNCODE']].rename(columns={'BEGIN':'TIME'})
begin_load_df['process'] = 1
end_load_df = sbatch[['END','RETURNCODE']].rename(columns={'END':'TIME'})
end_load_df['process'] = -1
load_df = begin_load_df.append(end_load_df)
load_df = load_df.sort_values('TIME').reset_index()

In [50]:
load_df['running'] = np.cumsum(load_df['process'])

In [53]:
load_df['running'].value_counts()

1      3605737
0      2505577
2      1691299
3       969452
4       645490
        ...   
540          2
539          2
538          2
537          2
556          1
Name: running, Length: 557, dtype: int64

In [61]:
load_df[(load_df['running'] > 400) & (load_df['RETURNCODE'] == 0)]

,index,TIME,RETURNCODE,process,running
1559996,393611,2020-11-10 08:51:46.496072,0,1,401
1559997,5157480,2020-11-10 08:51:46.499066,0,1,402
1559998,393612,2020-11-10 08:51:46.518310,0,1,403
1559999,5157481,2020-11-10 08:51:46.536698,0,1,404
1560000,393613,2020-11-10 08:51:46.539404,0,1,405
...,...,...,...,...,...
18918351,9503566,2021-10-02 08:15:36.984302,0,1,440
18918352,9503482,2021-10-02 08:15:36.987820,0,-1,439
18918353,4726722,2021-10-02 08:15:36.993135,0,1,440
18918354,9503567,2021-10-02 08:15:36.998663,0,1,441


In [62]:
load_df[(load_df['running'] > 400) & (load_df['RETURNCODE'] == 1)]

,index,TIME,RETURNCODE,process,running
1560116,5157192,2020-11-10 08:51:50.935295,1,-1,407
1560266,393447,2020-11-10 08:51:54.099137,1,-1,433
1560272,393448,2020-11-10 08:51:54.147870,1,-1,437
1560330,393449,2020-11-10 08:51:54.845623,1,-1,429
1560344,393450,2020-11-10 08:51:54.965285,1,-1,417
...,...,...,...,...,...
18918392,9503317,2021-10-02 08:15:38.817160,1,-1,405
18918393,9503316,2021-10-02 08:15:38.824229,1,-1,404
18918394,9503310,2021-10-02 08:15:38.825354,1,-1,403
18918395,9503318,2021-10-02 08:15:38.830735,1,-1,402


In [79]:
    1 - (len(load_df[load_df['RETURNCODE'] == 1]) / len(load_df))

0.8641893882667483

In [80]:
len(load_df)

19094826

In [72]:
for i in range(6):
    print(len(load_df[(load_df['running'] > i * 100) & 
                      (load_df['running'] < (i + 1) * 100) & 
                      (load_df['RETURNCODE'] == 0)]) / 
          len(load_df[(load_df['running'] > i * 100) & 
                      (load_df['running'] < (i + 1) * 100) & 
                      (load_df['RETURNCODE'] == 1)]))

5.861656179654965
3.793390319053772
5.381062069648784
4.621005171727887
8.135135135135135
29.875


In [5]:
sb99 = sbatch[(sbatch['BEGIN'].dt.month == 9) & (sbatch['BEGIN'].dt.day == 9)]

In [18]:
sb99[sb99['RETURNCODE'] == 1].shape


(4075, 4)

In [21]:
# sb99[sb99['RETURNCODE'] == 1].head()
start = 120
sb99.iloc[-20:]

,index,BEGIN,DURATION,RETURNCODE
25365,9286657,2021-09-09 23:57:28.203146,0.478626,0
25366,9286658,2021-09-09 23:57:28.414825,0.687578,0
25367,9286659,2021-09-09 23:57:28.416321,0.733490,0
25368,9286660,2021-09-09 23:57:35.864990,0.641414,1
25369,9286661,2021-09-09 23:57:36.419406,0.553860,1
25370,9286662,2021-09-09 23:57:38.369906,0.479344,0
25371,9286663,2021-09-09 23:57:52.621966,0.027462,0
25372,9286664,2021-09-09 23:57:52.793472,0.103359,0
25373,9286665,2021-09-09 23:59:15.264929,20.047867,1
25374,9286666,2021-09-09 23:59:15.294801,20.099541,1


In [22]:
sbatch[sbatch.index == 9286665]

,BEGIN,DURATION,RETURNCODE
9286665,2021-09-09 23:59:15.264929,20.047867,1


In [23]:
sbatch['value_grp'] = (sbatch.RETURNCODE.diff(1) != 0).cumsum()

In [39]:
sbatch

,index,BEGIN,DURATION,RETURNCODE,value_grp
0,0,2020-10-16 08:15:39.278699,0.073476,0,1
1,1,2020-10-16 08:18:08.313309,0.183632,0,1
2,2,2020-10-16 08:22:48.128689,0.075471,0,1
3,3,2020-10-16 08:25:13.257408,0.094844,0,1
4,4,2020-10-16 08:31:01.460723,0.074988,0,1
...,...,...,...,...,...
9547408,9547408,2021-10-07 21:59:35.014602,0.060087,0,393555
9547409,9547409,2021-10-07 21:59:35.238970,0.098044,0,393555
9547410,9547410,2021-10-07 21:59:57.265189,0.024550,0,393555
9547411,9547411,2021-10-07 22:00:04.024360,0.039419,0,393555


In [40]:
codes_grouped = pd.DataFrame({'Sindex' : sbatch.groupby('value_grp').index.first(),
            'BeginDate' : sbatch.groupby('value_grp').BEGIN.first(), 
              'EndDate' : sbatch.groupby('value_grp').BEGIN.last(),
              'Consecutive' : sbatch.groupby('value_grp').size(), 
              'Returncode' : sbatch.groupby('value_grp').RETURNCODE.first()})\
            .sort_values('Consecutive', ascending=False)

In [41]:
codes_grouped[codes_grouped['Returncode'] == 1]

,Sindex,BeginDate,EndDate,Consecutive,Returncode
value_grp,,,,,
183154,3004577,2021-05-18 06:32:07.287798,2021-05-18 06:39:33.080816,900,1
367986,8701097,2021-05-18 06:30:43.047964,2021-05-18 06:35:56.047756,876,1
46910,1011839,2020-12-04 15:44:23.627296,2020-12-04 15:47:21.847208,384,1
47000,1012571,2020-12-04 15:50:47.020123,2020-12-04 15:53:22.422054,348,1
284331,5772308,2020-11-30 10:17:07.565682,2020-11-30 10:18:08.074493,343,1
...,...,...,...,...,...
141377,2271316,2021-04-01 20:56:00.663857,2021-04-01 20:56:00.663857,1,1
141379,2271320,2021-04-01 20:56:00.712054,2021-04-01 20:56:00.712054,1,1
141381,2271398,2021-04-01 20:56:12.398793,2021-04-01 20:56:12.398793,1,1


In [45]:
mid = 3004577
data[3004577]
# data[mid-15:mid+15]

['2021-05-18',
 '06:32:07.287798',
 '-',
 'user',
 '9221',
 '-',
 'retry',
 '0',
 '-',
 'time',
 '19.054259061813354',
 '-',
 'returncode',
 '1',
 '-',
 'command',
 "['/usr/bin/scontrol',",
 "'show',",
 "'job']"]

In [46]:
with open('../data/slurm_wrapper_ce5.log') as f:
    data1 = ([line for line in f])
with open('../data/slurm_wrapper_ce6.log') as f:
    data1.extend([line for line in f])